# MGSC 404 Midterm Case Study

In [243]:
# pip install gurobipy

In [244]:
# Import packages
from gurobipy import *
import numpy as np
import pandas as pd

# ***Question 1***

In [245]:
model = Model('Funds Allocation')

#### Production Variables (IBM)

In [246]:
# For each quarter, add a decision variable representing the amount of IBM adapters produced
Qi_IBM_prod = model.addVars(range(1, 5), lb=0, name=[f"Q{i}_IBM_Prod" for i in range(1, 5)])
Qi_IBM_prod

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Production Variables (Mac)

In [247]:
# For each quarter, add a decision variable representing the amount of Mac adapters produced
Qi_Mac_prod = model.addVars(range(1, 5),lb=0, name=[f"Q{i}_Mac_Prod" for i in range(1, 5)])
Qi_Mac_prod

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Additional Sales from Ad (IBM)

In [248]:
# For each quarter, add a decision variable representing the additional amount of the IBM adapters sold after advertising
Qi_IBM_Ad = model.addVars(range(1, 5),lb=0, name=[f"Q{i}_IBM_Ad" for i in range(1, 5)])
Qi_IBM_Ad

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Additional Sales from Ad (Mac)

In [249]:
# For each quarter, add a decision variable representing the additional amount of the Mac adapters sold after advertising
Qi_Mac_Ad = model.addVars(range(1, 5), lb=0, name=[f"Q{i}_Mac_Ad" for i in range(1, 5)])
Qi_Mac_Ad

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Sales Variables (IBM)

In [196]:
# For each quarter, add a decision variable representing the sales of IBM adapters (units)
Qi_IBM_sales = model.addVars(range(1, 5),lb=0, name=[f"Q{i}_IBM_sales" for i in range(1, 5)])
Qi_IBM_sales

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Sales Variables (Mac)

In [197]:
# For each quarter, add a decision variable representing the sales of Mac adapters (units)
Qi_Mac_sales = model.addVars(range(1, 5), lb=0, name=[f"Q{i}_Mac_sales" for i in range(1, 5)])
Qi_Mac_sales

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

#### Advertising Cost

In [198]:
# For each quarter, add a decision variable representing the total advertising cost
advertising_cost = 10
Qi_Ad_cost = model.addVars(range(1,5), lb=0, name = [f"Q{i}_Ad_cost" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_Ad_cost[i] == advertising_cost * (Qi_IBM_Ad[i] + Qi_Mac_Ad[i]), name = f"Define_Q{i}_Ad_cost")

#### IBM Inventory Level

In [199]:
# For each quarter, add a decision variable representing the inventory level of IBM adapters for each quarter
# Qi_IBM_inventory[0] means the initial inventory level of IBM adapters at the beginning of 1996
# Qi_IBM_inventory[i] means the inventory level of IBM adapters at the end of the i-th quarter
Qi_IBM_inventory = model.addVars(range(1,5), lb=0, name = [f"Q{i}_IBM_inventory" for i in range(1,5)])
Qi_IBM_inventory[0] = 10000
for i in range(1,5):
    model.addConstr(Qi_IBM_inventory[i] == Qi_IBM_inventory[i-1] + Qi_IBM_prod[i] - Qi_IBM_sales[i] - Qi_IBM_Ad[i], name = f"Define_Q{i}_IBM_inventory")
Qi_IBM_inventory

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 0: 10000}

#### Mac Inventory Level

In [200]:
# For each quarter, add a decision variable representing the inventory level of Mac adapters for each quarter
# Qi_Mac_inventory[0] means the initial inventory level of Mac adapters at the beginning of 1996
# Qi_Mac_inventory[i] means the inventory level of Mac adapters at the end of the i-th quarter
Qi_Mac_inventory = model.addVars(range(1,5), lb=0, name = [f"Q{i}_Mac_inventory" for i in range(1,5)])
Qi_Mac_inventory[0] = 5000
for i in range(1,5):
    model.addConstr(Qi_Mac_inventory[i] == Qi_Mac_inventory[i-1] + Qi_Mac_prod[i] - Qi_Mac_sales[i] - Qi_Mac_Ad[i], name = f"Define_Q{i}_Mac_inventory")

#### Inventory Carry Cost

In [201]:
# We assume the inventory carry cost is compoundedly calculating
inventory_monthly_cost = 0.01
inventory_quarterly_cost = round(((1+inventory_monthly_cost) ** 3) - 1, 4) 
print(inventory_quarterly_cost)

0.0303


In [202]:
IBM_unit_cost = 100
Mac_unit_cost = 110
IBM_unit_price = 175
Mac_unit_price = 200

In [203]:
# For each quarter, add a decision variable representing the inventory carry cost for each quarter
# We use average inventory level to calculate the inventory carry cost
# average inventory level = inventory at the beginning of the quarter + inventory at the end of the quarter
# We asssume the inventory level at the beginning of the quarter is the inventory level at the end of the last quarter
Qi_carry_cost = model.addVars(range(1,5), lb=0, name=[f"Q{i}_carry_cost" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_carry_cost[i] == IBM_unit_cost * inventory_quarterly_cost * 0.5 * (Qi_IBM_inventory[i-1] + Qi_IBM_inventory[i]) +  # take average inventory
                                        Mac_unit_cost * inventory_quarterly_cost * 0.5 * (Qi_Mac_inventory[i-1] + Qi_Mac_inventory[i]), name = f"Define_Q{i}_carry_cost")

#### Variable Cost

In [204]:
# For each quarter, add a variable to the model representing variable cost ($) based on the units produced
Qi_variable_cost = model.addVars(range(1,5), lb=0, name = [f"Q{i}_variable_cost" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_variable_cost[i] == IBM_unit_cost * Qi_IBM_prod[i] + Mac_unit_cost * Qi_Mac_prod[i], name = f"Define_Q{i}_variable_cost")

#### Fixed Cost

In [205]:
quarterly_fixed_cost = 150000

#### Revenue

In [206]:
# For each quarter, add a variable to the model representing revenue generated based on total sales ($)
Qi_total_revenue = model.addVars(range(1,5), lb=0, name = [f"Q{i}_total_revenue" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_total_revenue[i] == IBM_unit_price * (Qi_IBM_sales[i] + Qi_IBM_Ad[i]) + Mac_unit_price * (Qi_Mac_sales[i] + Qi_Mac_Ad[i]), name = f"Define_Q{i}_revenus")

#### Initial Funds for 1996

In [207]:
receivables_IBM = {
    1: 0.75,
    2: 0.20,
    3: 0.03,
    4: 0.02 * 0.5  # The price goes down by a half when the adapters are uncollectible
}
receivables_Mac = {
    1: 0.80,
    2: 0.11,
    3: 0.05,
    4: 0.04 * 0.5 # The price goes down by a half when the adapters are uncollectible
}

In [208]:
IBM_sales_1995 = {
    1: 5000,
    2: 6000,
    3: 7000,
    4: 8000
}
Mac_sales_1995 = {
    1: 1000,
    2: 1200,
    3: 1400,
    4: 1600
}

We assume that the unit price for IBM adapters was the same in 1995 as in 1996.

In [209]:
# Calculating the amount of funds that came from the sales of IBM adapters in 1995 but was received in 1996
IBM_initial_funds = {}
IBM_initial_funds[1] = receivables_IBM[4] * IBM_unit_price * IBM_sales_1995[2] + receivables_IBM[3] * IBM_unit_price * IBM_sales_1995[3] + receivables_IBM[2] * IBM_unit_price * IBM_sales_1995[4]
IBM_initial_funds[2] = receivables_IBM[4] * IBM_unit_price * IBM_sales_1995[3] + receivables_IBM[3] * IBM_unit_price * IBM_sales_1995[4]
IBM_initial_funds[3] = receivables_IBM[4] * IBM_unit_price * IBM_sales_1995[4]
IBM_initial_funds[4] = 0

In [210]:
IBM_initial_funds

{1: 327250.0, 2: 54250.0, 3: 14000.0, 4: 0}

We assume that the unit price for Mac adapters was the same in 1995 as in 1996.

In [211]:
# Calculating the amount of cash that came from the sales of Mac adapters in 1995 but was received in 1996
Mac_initial_funds = {}
Mac_initial_funds[1] = receivables_Mac[4] * Mac_unit_price * Mac_sales_1995[2] + receivables_Mac[3] * Mac_unit_price * Mac_sales_1995[3] + receivables_Mac[2] * Mac_unit_price * Mac_sales_1995[4]
Mac_initial_funds[2] = receivables_Mac[4] * Mac_unit_price * Mac_sales_1995[3] + receivables_Mac[3] * Mac_unit_price * Mac_sales_1995[4]
Mac_initial_funds[3] = receivables_Mac[4] * Mac_unit_price * Mac_sales_1995[4]
Mac_initial_funds[4] = 0

In [212]:
Mac_initial_funds

{1: 54000.0, 2: 21600.0, 3: 6400.0, 4: 0}

#### Account Receivables in 1996

In [213]:
credit_interest_rate = 0.15
surplus_fund_rate = 0.06

In [214]:
# The function is used to calculate the account receivables for 1996
def calculate_receivables(sales, sales_Ad, price, proportion):
    receivables = {}
    for i in range(1,5):
        receivables[i] = (sales[i] + sales_Ad[i]) * price * proportion[1]
        for j in range(1,i):
            receivables[i] += (sales[i-j] + sales_Ad[i-j]) * price * proportion[j+1]
    return receivables

In [215]:
# For each quarter, define the amount of receivables from selling IBM adapters
Qi_IBM_receivables = model.addVars(range(1,5), lb=0, name = [f"Q{i}_IBM_receivables" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_IBM_receivables[i] == calculate_receivables(Qi_IBM_sales, Qi_IBM_Ad, IBM_unit_price, receivables_IBM)[i], name = f"Define_Q{i}_IBM_receivables")

In [216]:
# For each quarter, define the amount of receivables from selling Mac adapters
Qi_Mac_receivables = model.addVars(range(1,5), lb=0, name = [f"Q{i}_Mac_receivables" for i in range(1,5)])
for i in range(1,5):
    model.addConstr(Qi_Mac_receivables[i] == calculate_receivables(Qi_Mac_sales, Qi_Mac_Ad, Mac_unit_price, receivables_Mac)[i], name = f"Define_Q{i}_Mac_receivables")

#### Amount Borrowed, Interest Expense, Surplus Return

In [217]:
# Qi_amount_borrowed[i] represents the amount of credit we borrowed from the bank for the i-th quarter
Qi_amount_borrowed = model.addVars(range(1,5), lb=0, name = [f"Q{i}_amount_borrowed" for i in range(1,5)])

# Qi_amount_payback[i] represents the amount of funds we should pay back the bank for the i-th quarter
Qi_amount_payback = model.addVars(range(1,5), lb=0, name=[f"Q{i}_amount_payback" for i in range(1,5)]) 

# Qi_interest_expense[i] represents the amount of interest we should pay due to the borrowing for the i-th quarter
Qi_interest_expense = model.addVars(range(1,5), lb=0, name=[f"Q{i}_interest_expense" for i in range(1,5)])

# Qi_surplus_return[i] represents the amount of investment return we earn due to the investment of surplus fund to short-term government securities for the i-th quarter
Qi_surplus_return = model.addVars(range(1,5), lb=0, name=[f"Q{i}_surplus_return" for i in range(1,5)])

# Qi_cash[i] represents the amount of cash we have at the beginning of the i-th quarter
Qi_cash = model.addVars(range(1,5), lb=0, name=[f"Q{i}_cash" for i in range(1,5)])

# Qi_debt[i] represents the amount of debt which we should pay interest for in i-th quarter
Qi_debt = model.addVars(range(1,5), lb=0, name=[f"Q{i}_debt" for i in range(1,5)])

In [218]:
# The initial investment return from surplus fund, the initial interest expense, and the initial account receivables (for both IBM and Mac adapters) are all 0
Qi_surplus_return[0] = 0
Qi_interest_expense[0] = 0
Qi_IBM_receivables[0] = 0
Qi_Mac_receivables[0] = 0

In [219]:
# Add a constraint that defines the cash variable at the beginning of each quarter
for i in range(1,5):
    model.addConstr(Qi_cash[i] == Qi_surplus_return[i-1] + IBM_initial_funds[i] + Mac_initial_funds[i] + Qi_IBM_receivables[i-1] + Qi_Mac_receivables[i-1], name = f"Define_Q{i}_cash")

In [220]:
# Add a constraint that the money on hand (cash available + amount borrowed from the bank) should be greater or equal to the cost generated from production, advertising, inventory, and interest expenses
for i in range(1,5):
    model.addConstr(Qi_cash[i] + Qi_amount_borrowed[i] >= Qi_variable_cost[i] + quarterly_fixed_cost + Qi_carry_cost[i] + Qi_Ad_cost[i] + Qi_interest_expense[i-1], name = f"Q{i}_cost_amount_borrowed")

In [221]:
# Add a constraint that defines the amount of suplus return for each quarter
for i in range(1,5):
    model.addConstr(Qi_surplus_return[i] == (surplus_fund_rate/4) * (Qi_cash[i] + Qi_amount_borrowed[i] - (Qi_variable_cost[i] + quarterly_fixed_cost + Qi_carry_cost[i] + Qi_Ad_cost[i] + Qi_interest_expense[i-1])), name = f"Define_Q{i}_surplus_return")

In [222]:
# Add a constraint that defines the amount of debt (the portion that the company will pay interest for) for each quarter
model.addConstr(Qi_debt[1] == Qi_amount_borrowed[1], name = "Define_Q1_debt")
model.addConstr(Qi_debt[2] == Qi_amount_borrowed[1] + Qi_amount_borrowed[2] - Qi_amount_payback[1], name = "Define_Q2_debt")
model.addConstr(Qi_debt[3] == Qi_amount_borrowed[1] + Qi_amount_borrowed[2] + Qi_amount_borrowed[3] - Qi_amount_payback[1] - Qi_amount_payback[2], name = "Define_Q3_debt")
model.addConstr(Qi_debt[4] == Qi_amount_borrowed[1] + Qi_amount_borrowed[2] + Qi_amount_borrowed[3] + Qi_amount_borrowed[4]- Qi_amount_payback[1] - Qi_amount_payback[2] - Qi_amount_payback[3], name = "Define_Q4_debt")

<gurobi.Constr *Awaiting Model Update*>

In [223]:
# Add a constraint that define the amount interest expenses for each quarter
for i in range(1,5):
    model.addConstr(Qi_interest_expense[i] == (credit_interest_rate/4) * Qi_debt[i], name = f"Define_Q{i}_interest_expense")    

In [224]:
# Add a constraint that the amount we pay back to the bank should be less or equal to the amount of debt for each quarter
for i in range(1,5):
     model.addConstr(Qi_amount_payback[i] <= Qi_debt[i], name = f"Q{i}_payback_limit")

In [225]:
# Add a constraint that defines the maximum amount that we can borrow from the bank for each quarter
model.addConstr(Qi_amount_borrowed[1] <= 750000, name = "Q1_amount_borrowed")
model.addConstr(Qi_amount_borrowed[2] <= 750000 - Qi_amount_borrowed[1] + Qi_amount_payback[1], name = "Q2_amount_borrowed")
model.addConstr(Qi_amount_borrowed[3] <= 750000 - Qi_amount_borrowed[1] + Qi_amount_payback[1] - Qi_amount_borrowed[2] + Qi_amount_payback[2], name = "Q3_amount_borrowed")
model.addConstr(Qi_amount_borrowed[4] <= 750000 - Qi_amount_borrowed[1] + Qi_amount_payback[1] - Qi_amount_borrowed[2] + Qi_amount_payback[2] - Qi_amount_borrowed[3] + Qi_amount_payback[3], name = "Q4_amount_borrowed")

<gurobi.Constr *Awaiting Model Update*>

#### Maximum Production Constraints

IBM

In [226]:
# For each quarter, add a constraint stipulating that the actual production of IBM adapters cannot exceed the quarter's maximum production
# We assume the plant operates two shifts per day, and 12 hours per shift
# Operation Hours: 24h * 20 days = 480h/month = 1440h/quarter
IBM_max_production = {
    1: 43200, # 1440 * 30 = 43200
    2: 43200,
    3: 43200,
    4: 47520 # 1440 * 33 = 47520 (the production capacity increaes by 10% at the beginning of the 4th quarter)
}
for i in range(1,5):
    model.addConstr(Qi_IBM_prod[i] <= IBM_max_production[i], name = f"Q{i}_IBM_Max_Prod")

Mac

In [227]:
# For each quarter, add a constraint stipulating that the actual production of Mac adapters cannot exceed the quarter's maximum production
# We assume the plant operates two shifts per day, and 12 hours per shift
# Operation Hours: 24h * 20 days = 480h/month = 1440h/quarter
Mac_max_production = {
    1: 14400, # 1440 * 10 = 14400
    2: 14400,
    3: 14400,
    4: 15840 # 1440 * 11 = 15840 (the production capacity increaes by 10% at the beginning of the 4th quarter)
}
for i in range(1,5):
    model.addConstr(Qi_Mac_prod[i] <= Mac_max_production[i], name = f"Q{i}_Mac_Max_Prod")

#### Maximum Sales Constraints

IBM

In [228]:
# For each quarter, add a constraint stipulating that the actual sales of IBM adapters cannot exceed the quarter's maximum sales after advertising
IBM_sales_1996 = {
    1: 9000,
    2: 10000,
    3: 11000,
    4: 12000
}
for i in range(1,5):
    model.addConstr(Qi_IBM_sales[i] <= IBM_sales_1996[i] + Qi_IBM_Ad[i], name = f"Q{i}_IBM_Max_Sales")

Mac

In [229]:
# For each quarter, add a constraint stipulating that the actual sales of Mac adapters cannot exceed the quarter's maximum sales after advertising
Mac_sales_1996 = {
    1: 1800,
    2: 2000,
    3: 2200,
    4: 2400
}
for i in range(1,5):
    model.addConstr(Qi_Mac_sales[i] <= Mac_sales_1996[i] + Qi_Mac_Ad[i], name = f"Q{i}_Mac_Max_Sales")

#### Sales <= Inventory Constraints

IBM

In [230]:
# For each quarter, add a constraint that the total sales of IBM adapters after advertising should not exceed the inventory level at that quarter
for i in range(1,5):
    model.addConstr(Qi_IBM_sales[i] + Qi_IBM_Ad[i] <= Qi_IBM_inventory[i], name = f"Q{i}_IBM_Sales_Inventory")

Mac

In [231]:
# For each quarter, add a constraint that the total sales of Mac adapters after advertising should not exceed the inventory level at that quarter
for i in range(1,5):
    model.addConstr(Qi_Mac_sales[i] + Qi_Mac_Ad[i] <= Qi_Mac_inventory[i], name = f"Q{i}_Mac_Sales_Inventory")

#### Safety Inventory Constraints

IBM

In [232]:
# For each quarter, add a constraint that the production plus the end-of-period inventory for the IBM adapters should be at least 10% larger than the estimated sales for the quarter.
for i in range(1,5):
    model.addConstr(Qi_IBM_prod[i] + Qi_IBM_inventory[i-1] >= 1.1 * Qi_IBM_sales[i], 
                    name = f"Q{i}_IBM_safety_inventory")

Mac

In [233]:
# For each quarter, add a constraint that the production plus the end-of-period inventory for the Mac adapters should be at least 10% larger than the estimated sales for the quarter.
for i in range(1,5):
    model.addConstr(Qi_Mac_prod[i] + Qi_Mac_inventory[i-1] >= 1.1 * Qi_Mac_sales[i] , 
                    name = f"Q{i}_Mac_safety_inventory")

#### Inventory Transition Constraint

IBM

In [234]:
# Add a constraints that the inventory level of IBM adapters plus production at the end of the fourth quarter of 1996 should be at least twice the maximum expected sales for that quarter
model.addConstr(Qi_IBM_inventory[4] + Qi_IBM_prod[4] >= 2 * IBM_sales_1996[4], name=f"IBM_transition")

<gurobi.Constr *Awaiting Model Update*>

Mac

In [235]:
# Add a constraints that the inventory level of Mac adapters plus production at the end of the fourth quarter of 1996 should be at least twice the maximum expected sales for that quarter
model.addConstr(Qi_Mac_inventory[4] + Qi_Mac_prod[4] >= 2 * Mac_sales_1996[4], name=f"Mac_transition")

<gurobi.Constr *Awaiting Model Update*>

# ***Question 2***

#### Profit

In [236]:
profit = 0
for i in range(1,5):
    revenue = Qi_total_revenue[i] + Qi_surplus_return[i]
    cost = Qi_variable_cost[i] + quarterly_fixed_cost + Qi_carry_cost[i] + Qi_Ad_cost[i] + Qi_interest_expense[i]
    profit += revenue - cost
print(profit)

-600000.0 + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var

#### Objective Function

In [237]:
model.setObjective(profit, GRB.MAXIMIZE)

#### Run the Model

In [238]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 94 rows, 80 columns and 337 nonzeros
Model fingerprint: 0x0fbb5a30
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 8e+05]
Presolve removed 49 rows and 40 columns
Presolve time: 0.01s
Presolved: 45 rows, 40 columns, 279 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4300000e+31   6.861417e+31   2.430000e+01      0s
      28    4.4741978e+06   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.474197806e+06


In [239]:
print("Optimal Obj Value:", model.objVal)

Optimal Obj Value: 4474197.80620049


#### Sensitivity Analysis Tables

In [240]:
def sensitivity_analysis_table(model: Model) -> pd.DataFrame:
    # Ensure the model has been solved
    if model.Status != 2:  # Model status 2 is 'Optimized'
        raise ValueError("Model must be optimized before sensitivity analysis can be performed.")
    
    # Collecting sensitivity analysis for variables
    variables_data = {
        'Variable': [],
        'Optimal Value': [],
        'Reduced Cost': [],
        'Allowable Increase (Obj Coeff)': [],
        'Allowable Decrease (Obj Coeff)': []
    }
    for v in model.getVars():
        variables_data['Variable'].append(v.VarName)
        variables_data['Optimal Value'].append(v.X)
        variables_data['Reduced Cost'].append(v.RC)
        variables_data['Allowable Increase (Obj Coeff)'].append(v.SAObjUp)
        variables_data['Allowable Decrease (Obj Coeff)'].append(v.SAObjLow)
    
    variables_df = pd.DataFrame(variables_data)
    
    # Collecting sensitivity analysis for constraints
    constraints_data = {
        'Constraint': [],
        'RHS Constraint':[],
        'Slack': [],
        'Final Value':[],
        'Shadow Price': [],
        'Allowable Increase (RHS)': [],
        'Allowable Decrease (RHS)': []
    }
    for c in model.getConstrs():
        constraints_data['Constraint'].append(c.ConstrName)
        constraints_data['RHS Constraint'].append(c.RHS)
        constraints_data['Slack'].append(c.Slack)
        constraints_data['Final Value'].append(c.RHS - c.Slack)
        constraints_data['Shadow Price'].append(c.Pi)
        constraints_data['Allowable Increase (RHS)'].append(c.SARHSUp)
        constraints_data['Allowable Decrease (RHS)'].append(c.SARHSLow)
    
    constraints_df = pd.DataFrame(constraints_data)
 
    # Return both DataFrames separately
    return variables_df, constraints_df

variables_df, constraints_df = sensitivity_analysis_table(model)

In [241]:
pd.set_option('display.max_rows', variables_df.shape[0]+1)
variables_df

,Variable,Optimal Value,Reduced Cost,Allowable Increase (Obj Coeff),Allowable Decrease (Obj Coeff)
0,Q1_IBM_Prod,0.000000e+00,-2.354837,2.354837,-inf
1,Q2_IBM_Prod,6.973550e+03,0.000000,2.424315,-53.780680
2,Q3_IBM_Prod,1.682166e+04,0.000000,1.903405,-26.354820
3,Q4_IBM_Prod,1.219859e+04,0.000000,14.768862,-11.420428
4,Q1_Mac_Prod,8.316414e+03,0.000000,4215.578722,-2.648749
5,Q2_Mac_Prod,1.440000e+04,0.000000,inf,-2.726899
6,Q3_Mac_Prod,1.440000e+04,0.000000,inf,-2.089137
7,Q4_Mac_Prod,0.000000e+00,-15.033249,15.033249,-inf
8,Q1_IBM_Ad,0.000000e+00,-22.231710,22.231710,-inf
9,Q2_IBM_Ad,0.000000e+00,-16.941193,16.941193,-inf


In [242]:
pd.set_option('display.max_rows', constraints_df.shape[0]+1)
constraints_df

,Constraint,RHS Constraint,Slack,Final Value,Shadow Price,Allowable Increase (RHS),Allowable Decrease (RHS)
0,Define_Q1_Ad_cost,0.0,0.000000e+00,0.000000e+00,-2.223171,6.850485e+04,-2.483895e+04
1,Define_Q2_Ad_cost,0.0,0.000000e+00,0.000000e+00,-1.694119,7.322309e+04,-4.264552e+04
2,Define_Q3_Ad_cost,0.0,0.000000e+00,0.000000e+00,-1.293296,8.347719e+04,-5.466742e+04
3,Define_Q4_Ad_cost,0.0,0.000000e+00,0.000000e+00,-1.015000,2.541681e+06,-1.916138e+04
4,Define_Q1_IBM_inventory,10000.0,0.000000e+00,1.000000e+04,219.962265,1.083398e+04,9.434170e+03
5,Define_Q2_IBM_inventory,0.0,0.000000e+00,0.000000e+00,169.411935,1.079242e+03,-7.322309e+02
6,Define_Q3_IBM_inventory,0.0,0.000000e+00,0.000000e+00,129.329645,1.230378e+03,-8.347719e+02
7,Define_Q4_IBM_inventory,0.0,0.000000e+00,0.000000e+00,91.654092,5.957814e+02,-3.540422e+04
8,Define_Q1_Mac_inventory,5000.0,0.000000e+00,5.000000e+03,244.548812,5.917908e+03,4.377229e+03
9,Define_Q2_Mac_inventory,0.0,0.000000e+00,0.000000e+00,189.080027,1.669439e+03,-1.132661e+03


# ***Question 3***

### (a) What happens if the prices for the adapters are a little weak and they decrease to \\$173 for the IBM version and \\$198 for the Macintosh version? Does this make any difference? 


Yes, the optimal objective value (i.e. profit) decreases to \\$4,307,426 from \\$4,474,198 due to a decrease in revenue.


### (b) What about decreases to \\$172 and \\$197, respectively, for the IBM and Macintosh versions? Explain the answers in terms that Miller will understand. 


The optimal objective value (i.e. profit) further decreases to $4,224,846. This is because pricing is weaker than expected, meaning less revenue will be generated per adapter unit sold.


### (c) Suppose that American Office Systems can increase the price of the adapters to \\$179 and \\$204. How would this affect the original solution? 


In this case, the optimal objective value (i.e. profit) increases to $4,814,348 because more revenue is generated per unit sold.


# ***Question 4***

### (a) Suppose that an error was made in determining the costs of the adapters and that they really should have been \\$101 for the IBM version and \\$111 for the Macintosh version. What is the effect of this error? 


The optimal objective value (profit) decreases from \\$4,474,198 to \\$4,364,883 because of the higher production cost. With it, the production number of IBM decreased in Q2 and Q3. And the production number of Macintosh decreased in Q1. 


### (b) What about costs of \\$103 and \\$115? Explain the answers in terms that Miller will understand.


The optimal objective value (profit) decreased further from \\$4,474,198 to \\$4,042,801 because of the further higher production cost. Since each adapter costs more to produce, the company will generate less profit per adapter sold.


# ***Question 5***

### (a) Analyze the effect of this new collection policy and make a recommendation to Howell about whether to implement the new procedure. As before, any accounts receivable not collected by the end of the third quarter will be sold to a collection agency for $0.50 on the dollar. 


No, Howell shouldn’t implement the new procedure, because there’s a decrease in the objective function's optimal value going from \\$4,474,198 to \\$4,236,074 with the new model from the new collection policy and the additional cost splitted into quarterly fixed costs.


### (b) Howell wonders whether switching to selling adapters for all cash is worth the effort. This would ameliorate the cash squeeze because it would eliminate not only the slow collections but also the use of the collection agency for accounts that remain unpaid after 9 months. It would cost about $90,000 more than at present to implement the all-cash policy because the accounting system would need to be modified, and personnel would have to be retrained. Analyze this possibility and make a recommendation to Howell. 


Considering 1995’s collections table are as table 4, a 100% collection for accounts in 1996 would be beneficial because the objective function’s optimal value would go from the previous \\$4,236,074 yearly profit to \\$5,488,628 even with the additional cost. As a result, Howell should switch to selling adapters for all profit.

# ***Question 6***

### (a) Suppose that Williams overestimated the effectiveness of advertising. It now appears that $100 is needed to increase sales by one adapter. How will this affect the original solution? Explain the answer in terms that Miller will understand. 


The optimal objective value (i.e. profit) decreases to $3,474,891. This is because it now costs 10x more money to gain each additional sale through marketing. For this reason, less funds are allocated to marketing, meaning less demand created for the adapters and therefore decreased sales. We can observe this as the amount of advertisements purchased decreases from ~13,900 to ~7,400 and the number of sales decreases proportionally.


### (b) What happens if the required advertising outlay is $12.50 per additional adapter sold? 


In this case, the optimal objective value (i.e. profit) only decreases to \\$4,420,898 (from \\$4,474,198).

# ***Question 7***

### Suppose that the line of credit from Citibank that Howell thought he had arranged did not work out because of the poor financial situation of the company. The company can obtain one for the same amount from a small local bank; however, the interest rate is much higher, 24%. Analyze how this change affects American Office Systems. 


The optimal objective value (i.e. profit) decreases to \\$4,406,657 (from \\$4,474,198). This is the interest expense per dollar borrowed increases, decreasing the profit. This however does not increase the amount that the firm chooses to borrow from the line of credit as it is still optimal to withdraw the entire amount.


# ***Question 8***

### (a) Suppose that Garcia finds a bug in his original inventory model. Correcting it results in a safety cushion of 15% instead of the 10% he suggested previously. Determine whether this is important.


The allowable increase for this constraint is greater than 0.05 for every given quarter and so this bug does not change the optimal objective value of profit.


### (b) What if the error is 20%? Explain the answers in terms that Miller will understand. 


Once again the allowable increase for this constraint is greater than 0.1 for every given quarter and so this bug does not change the optimal objective value of profit. In other words, the model output says that it is worth it to produce more inventory for both products in each quarter than required by the safety cushion of 20%.


# ***Question 9***

### (a) Production capacity is scheduled to increase by 10% in the fourth quarter. Suppose that Miller is advised by the construction company that the work will not be finished until the following year. How will this delay affect the company’s plans? 


The company does not produce any Mac adapters in Q4 and only produces ~12,200 IBM adapters in Q4 (26% of the 47,520 original capacity) so this change does not affect any of the company’s plans nor does it affect any optimal values.


### (b) In addition to the delay in part (a), suppose that an accident in the production facility damages some of the equipment so that the capacity is decreased by 10% in the fourth quarter. Analyze how this will affect the original solution. 


Once again, the company does not produce any Mac adapters in Q4 and only produces ~12,200 IBM adapters in Q4 (28% of the 43,200 non-renovated capacity) so this change does not affect any of the company’s plans nor does it affect any optimal values.


# ***Question 10***

### Williams is concerned about the accuracy of Lu’s 1996 maximum expected sales forecasts. If errors in these forecasts have a significant impact on company profits, she is considering hiring a San Francisco marketing research firm for a more detailed analysis, which would charge $50,000 for the study. Analyze the potential impact if Lu’s forecasts are off by 1,000 for IBM adapters and 200 for Macintosh adapters each quarter. Should she hire the marketing research firm? 


The maximum sales is only a binding constraint for IBM adapters in Q3 where the shadow price is \\$6.467 which will lead to a decrease in profit of \\$6,467. The shadow price for Mac max sales are \\$11.11, \\$8.47, \\$6.47, and \\$5.08 respectively for quarters 1-4. Taking into account both the decreases in Mac and IBM maximum sales, the impact to profit would be \\$12,693 which is much lower than the \\$50,000 needed to do the study. For this reason Williams should not hire the marketing research firm. All changes are within the allowable decrease so the model does not need to be rerun with new constraints.


# ***Question 11***

### (a) To determine whether the extra $1 million line of credit is needed, analyze its effect on the original solution given in Question 2. 

It increases the original objective optimal value from \\$4,474,197 to \\$6,460,996. Additionally, the extra million dollars is fully borrowed in the first three quarters which increases production and ads due to the increase in funds.



### (b) To fully understand the ramifications of the extra $1,000,000 line of credit, redo: (i) Question 3b, (ii)Question 4b, (iii) Question 6a, and (iv) Question 8b. Summarize your results. 


Q3b (decrease adapters prices): Optimal Obj. Value decreases from \\$6,460,996 to \\$6,087,549

Q4b (increase adapters costs): Optimal Obj. Value decreases from \\$6,460,996 to \\$5,795,089

Q6a (decrease in ads returns): Optimal Obj. Value decreases from \\$6,460,996 to \\$3,821,656

Q8b (increase/error safety inventory cushion): Optimal Obj. Value stays the same, \\$6,460,996 to \\$6,460,996.

Results: 
An extra \\$1,000,000 line follows the dynamics of the previous model but with a higher base optimal objective value. Any decrease in adapter prices, increase in adapter costs or decrease in advertisement returns decreases the optimal objective value. As expected, the increase in the safety inventory cushion doesn’t affect the optimal objective value if it is still within the allowable increase.


### (c) What about Howell’s claim that an extra $8,000,000 line of credit is necessary? Use that adjustment and redo Question 6a.

Optimal Obj. Value with extra \\$8M line of credit: \\$9,798,131.
Q6a (decrease in ads returns): Optimal Obj. Value decreases from \\$9,798,131 to \\$4,700,512.

We can notice that although extra funds in the line of credit can increase the optimal objective value, there is a limit to it because of allowable increases, which means that \\$8,000,000 might be an excessive amount because we can see that the shadow prices for borrowing constraints are non binding (=0) at that point. Also, decreasing advertisement returns by 10x, decreases the optimal objective value by a lot more than previous lines of credit (\\$9,798,131 to \\$4,700,512) because of the increased borrowing.


# END OF REPORT